In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from transformers import AutoTokenizer, PreTrainedTokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments, AutoModelForCausalLM, DataCollatorForLanguageModeling
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

Mounted at /content/drive


In [ ]:
# tale.txt 파일 읽기
txt_file_path = "/content/drive/MyDrive/Tale/tale.txt"
with open(txt_file_path, "r", encoding="utf-8") as file:
    text = file.read()

# 일부분 출력
print(len(text))  # 처음 500자 출력

716655


In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='<s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
import accelerate
def process_text(text, tokenizer):
    sentences = text.split('\n')
    processed_sentences = []
    for sentence in sentences:
        if sentence.strip():
            processed_sentence = '<s> ' + sentence.strip() + ' </s>'
            processed_sentences.append(processed_sentence)
    tokenized_input = tokenizer(processed_sentences, return_tensors='pt', padding=True, truncation=True, max_length=1024)
    return tokenized_input

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, text, tokenizer):
        self.tokenized_input = process_text(text, tokenizer)

    def __len__(self):
        return self.tokenized_input['input_ids'].size(0)

    def __getitem__(self, idx):
        input_ids = self.tokenized_input['input_ids'][idx]
        attention_mask = self.tokenized_input['attention_mask'][idx]
        return {'input_ids': input_ids, 'attention_mask': attention_mask}


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/kogpt2-story-generator',
    local_rank=0,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_steps=500,  # 매 500 스텝마다 로깅
)



dataset = CustomDataset(text, tokenizer)
train_dataloader = DataLoader(dataset, batch_size=training_args.per_device_train_batch_size, shuffle=True, num_workers=4)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    accelerator="gpu",  # GPU 사용 지정
    devices=None,  # 사용 가능한 모든 GPU 활용
)

# Start training
trainer.train()

# Save the trained model
trainer.save_model()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


TypeError: Trainer.__init__() got an unexpected keyword argument 'accelerator'

In [ ]:
import accelerate

def process_text(text, tokenizer):
    sentences = text.split('\n')
    processed_sentences = []
    for sentence in sentences:
        if sentence.strip():
            processed_sentence = '<s> ' + sentence.strip() + ' </s>'
            processed_sentences.append(processed_sentence)
    tokenized_input = tokenizer(processed_sentences, return_tensors='pt', padding=True, truncation=True, max_length=1024)
    return tokenized_input

class CustomDataset(Dataset):
    def __init__(self, text, tokenizer):
        self.tokenized_input = process_text(text, tokenizer)

    def __len__(self):
        return self.tokenized_input['input_ids'].size(0)

    def __getitem__(self, idx):
        input_ids = self.tokenized_input['input_ids'][idx]
        attention_mask = self.tokenized_input['attention_mask'][idx]
        return {'input_ids': input_ids, 'attention_mask': attention_mask}

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/kogpt2-story-generator',
    local_rank=0,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_steps=500,
)

dataset = CustomDataset(text, tokenizer)
train_dataloader = DataLoader(dataset, batch_size=training_args.per_device_train_batch_size, shuffle=True, num_workers=4)

accelerator = accelerate.Accelerator(split_batches=True)

trainer = Trainer(
    model=accelerator.prepare(model),
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,  # 여기를 수정
    optimizers=(None, None),
)

trainer.train(resume_from_checkpoint=None)

accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(trainer.model)
accelerator.save_state(output_dir)

ModuleNotFoundError: No module named 'accelerate'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import pipeline

# Initialize text generation pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model="/content/drive/MyDrive/kogpt2-story-generator",  # Path to the trained model
    tokenizer="skt/kogpt2-base-v2",
    device=0  # Set device=0 to use GPU, -1 to use CPU
)

# Generate a chapter
generated_chapter = text_generation_pipeline(
    "숲 속 마을에는 용감한 다람쥐 토토가 살고 있습니다. 어느 날, 토토는 숲에서 반짝이는 빛을 따라가다가 작고 귀여운 요정 피피를 만나게 됩니다. 피피는 토토에게 마법의 숲 깊숙이 있는 보물에 대해 이야기해줍니다. 이 보물은 마을을 지키는 힘을 가지고 있다고 합니다. 토토는 피피와 친구가 되어 함께 보물을 찾기로 결심합니다.",
    max_length=500,
    min_length=300,
    num_return_sequences=1,
    truncation=True
)

print(generated_chapter[0]['generated_text'])
